In [1]:
%run stdPackages.ipynb
slides = True # print to slides format if True
out_folder = os.path.join(d['curr'], 'Misc', 'Figs')
d['data'] = os.path.join(d['curr'],'Misc','Data')
read = {'variables': ['Fundamentals', 'Load', 'Generators_Other'], 
        'variable2D': ['Generators_FuelMix','HourlyVariation'],
        'scalars': ['Scalars'],
        'maps': ['Generators_Categories']}
db = dbFromWB(os.path.join(d['data'],'mBasicInt1.xlsx'), read)
readSets(db)
from databaseAux import appIndexWithCopySeries

In [2]:
m = mBasicInt.mSimple(db)
rng = np.random.default_rng(seed=103)

## An energy system model with damages from emissions

Consider an optimization problem on the form:
$$\begin{align}
    \max\mbox{ } W &= \sum_{h} \left( \sum_l u\cdot D_{l,h}-\sum_{i} c_i E_{i,h} \right) - \sum_l \gamma\cdot I_l^{-\eta}\cdot\text{CO}_2 \\ 
    \text{s.t. }\sum_l D_{l,h} &= \sum_i E_{i,h} \\
    \text{CO}_2&\equiv \sum_j \phi_j \sum_i \mu_i^j \sum_h E_{i,h}\\
    E_{i,h}&\in[0,q_{i,h}] \\ 
    D_{l,h}&\in[0, L_{l,h}] 
\end{align}$$

where
* $h$ indicates hours,
* $l$ indicates different individuals,
* $i$ indicates different electricity technologies,
* $j$ indicates fuel types (oil, coal, gas etc.),
* $D_{l,h}$ is the consumption of energy of agent $l$ in hour $h$,
* $E_{i,h}$ is the production of energy by plant $i$ in hour $h$,
* $c_i$ is the marginal cost of energy production by plant $i$,
* $q_{i,h}$ is the hourly production capacity for plant $i$ in hour $h$,
* $L_{l,h}$ is the hourly constraint on the load for consumer $l$ in hour $h$,
* $\phi_j$ is the emission intensity of fuel type $j$, 
* $\mu_i^j$ is the fuel intensity of type $j$ for plant $i$,
* $I_l$ is the income level for agent $l$,
* $\eta,\gamma$.

Define the auxiliary function:
$$\begin{align}
    \Gamma_{i} = \sum_l \gamma I_l^{-\eta} \sum_{j} \phi_j\cdot \mu_i^j,
\end{align}$$
such that the welfare function can be written as:
$$\begin{align}
    W = \sum_h\left(\sum_l u \cdot D_{l,h} - \sum_i\left(c_i+\Gamma_{i}\right)E_{i,h}\right)
\end{align}$$

*Draw income distribution and compute $\Gamma$:*

In [3]:
N = 1000 # number of draws
γ = .01  # scale in damages
η = 2  # curvature, damages in income
μ, σ =0, 1 # parameters in distribution
def draw(μ, σ, N):
    return rng.lognormal(μ,σ,size=N)
def damage(γ, η, μ, σ, N, I = None, newSample = True):
    return γ * np.power(draw(μ, σ, N), -η) if newSample is True else γ * np.power(I, -η)
def Γ(γ, η, μ, σ, N, db, I = None, newSample = True):
    return sum(damage(γ, η, μ, σ, N, I = I, newSample = newSample)) * mBasicInt.plantEmissionIntensity(db).xs('CO2',level='EmissionType')

*Draw damages once, and then re-use the same sample:*

In [4]:
I = draw(μ,σ,N)

*This keeps returning the same result:*

In [5]:
Γ(γ, η, μ, σ, N, db, I = I, newSample = False) 

id
id1    7.838676
id2    4.559248
id3         0.0
id4         0.0
dtype: object

*Change a parameter and re-run:*

Add this to the cost structure:

In [6]:
kwargs = {'c': [{'variableName': 'Generation', 'parameter': lpCompiler.broadcast(Γ(γ, η, μ, σ, N, m.db, I=I, newSample=False), m.db['h'])}]}

*Add this as *initBlocks* to include it in the solve:*

In [7]:
m.solve(initBlocks=kwargs)

Solution status 0: Optimization terminated successfully. (HiGHS Status 7: Optimal)


In [8]:
m.db['Generation']

h  id 
1  id1     5.00
   id2     0.00
   id3     0.00
   id4    35.00
2  id1    50.00
   id2    30.00
   id3    22.50
   id4    17.50
3  id1    50.00
   id2    30.00
   id3    60.00
   id4     8.75
4  id1    12.00
   id2     0.00
   id3     0.00
   id4    28.00
dtype: float64

*Adjust a parameter and re-run:*

In [9]:
η = 4
kwargs = {'c': [{'variableName': 'Generation', 'parameter': lpCompiler.broadcast(Γ(γ, η, μ, σ, N, m.db, I=I, newSample=False), m.db['h'])}]}
m.solve(initBlocks=kwargs)

Solution status 0: Optimization terminated successfully. (HiGHS Status 7: Optimal)


In [10]:
m.db['Generation']

h  id 
1  id1     0.00
   id2     0.00
   id3     5.00
   id4    35.00
2  id1     0.00
   id2     0.00
   id3    60.00
   id4    17.50
3  id1     0.00
   id2     0.00
   id3    60.00
   id4     8.75
4  id1     0.00
   id2     0.00
   id3    12.00
   id4    28.00
dtype: float64